In [57]:
import os
import sys
import time
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import collections
import random

from cloudvolume import CloudVolume
from cloudvolume.lib import Vec, Bbox
from skimage.transform import resize

class Stack():
  """Collection of CloudVolume paths, with load & save operations
  """

  def __init__(self, paths):
    self.paths = paths

  def load(self, key, bbox, src_mip=0, dst_mip=0):
    src_cv = CloudVolume(self.paths[key], mip=src_mip, fill_missing=True)
    src_bbox = Bbox.from_slices(src_cv.slices_from_global_coords(bbox))
    img = src_cv[src_bbox.to_slices()][:,:,0,0]
    if dst_mip != src_mip:
        dst_cv = CloudVolume(self.paths[key], mip=dst_mip, fill_missing=True)
        dst_bbox = Bbox.from_slices(dst_cv.slices_from_global_coords(bbox))
        img = resize(img, dst_bbox.size3()[:2], preserve_range=True)
    return img

  def save(self, key, mip, bbox, img, is_field=False):
    """Save vector field to CloudVolume
    
    Args:
    * cv: CloudVolume object
    * mip: int representing mip-level of the CloudVolume
    * bbox: Bbox at mip 0 with size3 of (x,y,1)
    * img: nxm int64 ndarray with shape (x,y)
    * is_field: boolean whether img is a field object
        field object is 4d tensor with shape (1,x,y,2)
    """
    src_cv = CloudVolume(self.paths['image'], mip=mip)
    info = src_cv.info
    if is_field:
        info['data_type'] = 'float32'
        info['num_channels'] = 2
    cv = CloudVolume(self.paths[key], mip=mip, bounded=False, 
                     info=info, fill_missing=True, 
                     non_aligned_writes=True, cdn_cache=False)
    cv.commit_info()
    
    if is_field:
        s = cv.slices_from_global_coords(bbox) 
        s += (slice(0,2),)
        cv[s] = img.permute(1,2,0,3).data.cpu().numpy()
    else:
        s = cv.slices_from_global_coords(bbox)
        cv[s] = img[:,:,np.newaxis,np.newaxis]

class Optimizer():
  """
  
  Attributes:
    * src_image: nxm float64 ndarry normalized between [0,1]
      This is the image to be transformed by the returned vector field.
    * dst_images: list of nxm float64 ndarrays normalized between [0,1]
      This is the set of images that transformed src_image will be compared to.
    * src_mask_smooth: nxm float64 ndarray normalized between [0,1]
      The weight represents that degree to which a pixel participates in smooth
      deformation (0: not at all; 1: completely). This mask is used to reduce 
      the smoothness penalty for pixels that participate in a non-smooth 
      deformation.
    * src_mask_loss: nxm float64 ndarray normalized between [0,1]
      This mask is transformed by the vector field and used to reduce the MSE
      for pixels that participate in a defect.
    * dst_masks_loss: list of nxm float64 ndarrays normalized between [0,1]
      Exactly like the src_mask_loss above. These masks are only used to reduce the 
      MSE for pixels that participate in a defect.
  """
    
  def __init__(self, stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip):
    self.stack = stack
    self.bbox = bbox
    self.image_mip = image_mip
    self.output_mip = output_mip
    self.src_image = stack.load('image', bbox, src_mip=image_mip, dst_mip=image_mip) / 255.0
    self.src_fold = stack.load('fold', bbox, src_mip=fold_mip, dst_mip=image_mip) / 255.0
    self.src_crack = stack.load('crack', bbox, src_mip=crack_mip, dst_mip=image_mip) / 255.0
    self.src_smooth_mask = np.minimum(self.src_fold, self.src_crack)
    self.src_loss_mask = (stack.load('loss', bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
    
    self.dst_images = []
    self.dst_masks = []
    for z in dst_increment:
        dst_bbox = self.bbox + [0,0,z]
        dst_image = stack.load('image', dst_bbox, src_mip=image_mip, dst_mip=image_mip)
        self.dst_images.append(dst_image)
        dst_mask = (stack.load('loss', dst_bbox, src_mip=loss_mip, dst_mip=image_mip) == 0) * 1.0
        self.dst_masks.append(dst_mask)
        if output_mip != image_mip:
            dst_image = stack.load('image', dst_bbox, src_mip=output_mip, dst_mip=output_mip)
        self.stack.save('output', output_mip, dst_bbox, dst_image, is_field=False)
    
    self.identities = {}
        
  @staticmethod
  def center(var, dims, d):
      if not isinstance(d, collections.Sequence):
          d = [d for i in range(len(dims))]
      for idx, dim in enumerate(dims):
          if d[idx] == 0:
              continue
          var = var.narrow(dim, int(d[idx]/2), int(var.size()[dim] - d[idx]))
      return var

  def get_identity_grid(self, dim, cache=True):
      if dim not in self.identities:
          gx, gy = np.linspace(-1, 1, dim), np.linspace(-1, 1, dim)
          I = np.stack(np.meshgrid(gx, gy))
          I = np.expand_dims(I, 0)
          I = torch.FloatTensor(I)
          I = torch.autograd.Variable(I, requires_grad=False)
          I = I.permute(0,2,3,1).cuda()
          self.identities[dim] = I
      if cache:
          return self.identities[dim]
      else:
          return self.identities[dim].clone()

  def jacob(self, fields):
      def dx(f):
          p = Variable(torch.zeros((1,1,f.size(1),2))).cuda()
          return torch.cat((p, f[:,2:,:,:] - f[:,:-2,:,:], p), 1)
      def dy(f):
          p = Variable(torch.zeros((1,f.size(1),1,2))).cuda()
          return torch.cat((p, f[:,:,2:,:] - f[:,:,:-2,:], p), 2)
      fields = sum(map(lambda f: [dx(f), dy(f)], fields), [])
      field = torch.sum(torch.cat(fields, -1) ** 2, -1)
      return field

  def penalty(self, fields, mask=1):
      jacob = self.jacob(fields)
      jacob = torch.mul(jacob, mask)
      return torch.sum(jacob)

  def render(self, field):
      src = stack.load('image', bbox, src_mip=self.output_mip, dst_mip=self.output_mip) / 255.0
      src = torch.FloatTensor(src).cuda()
      src = Variable(src).unsqueeze(0).unsqueeze(0)
      if self.output_mip < self.image_mip:
          upsample = lambda x: nn.Upsample(scale_factor=2**x, mode='bilinear') if x >0 else (lambda y: y)
          field = upsample(self.image_mip - self.output_mip)(field.permute(0,3,1,2)).permute(0,2,3,1)
      y = F.grid_sample(src, field + self.get_identity_grid(field.size(2)))
      img = y.data.cpu().numpy()
      img = (img[0,0,:,:]*255).astype(np.uint8)
      self.stack.save('output', self.output_mip, self.bbox, img, is_field=False)

  def process(self, field=None, crop=0, write_field=True, ndownsamples=4, 
                                   currn=5, avgn=20, lambda1=0.4, lr=0.2, 
                                   eps=0.01, min_iter=20, max_iter=1000):
    """Compute vector field that minimizes mean squared error (MSE) between 
    transformed src_image & all dst_images regularized by the smoothness of the 
    vector field subject to masks that allow the vector field to not be smooth. 
    The minimization uses SGD.

    Args:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
      This field represents the derived vector field that transforms the 
      src_image subject to the contraints of the minimization.
    
    Returns:
    * field: nxmx2 float64 torch tensor normalized between [0,1]
    """
    downsample = lambda x: nn.AvgPool2d(2**x,2**x, count_include_pad=False) if x > 0 else (lambda y: y)
    upsample = nn.Upsample(scale_factor=2, mode='bilinear')
    s = torch.FloatTensor(self.src_image)
    src_var = Variable((s - torch.mean(s)) / torch.std(s)).cuda().unsqueeze(0).unsqueeze(0)
    src_smooth_mask_var = Variable(torch.FloatTensor(self.src_smooth_mask)).cuda().unsqueeze(0).unsqueeze(0)
    src_loss_mask_var = Variable(torch.FloatTensor(self.src_loss_mask)).cuda().unsqueeze(0).unsqueeze(0)
    dst_vars = []
    dst_mask_vars = []
    for d, m in zip(self.dst_images, self.dst_masks):
      d = torch.FloatTensor(d)
      dst_var = Variable((d - torch.mean(d)) / torch.std(d)).cuda().unsqueeze(0).unsqueeze(0)
      dst_vars.append(dst_var)
      mask_var = Variable(torch.FloatTensor(m)).cuda().unsqueeze(0).unsqueeze(0)
      dst_mask_vars.append(mask_var)
      
    dim = int(src_var.size()[-1] / (2 ** (ndownsamples - 1)))
    if field is None:
        field = Variable(torch.zeros((1,dim,dim,2))).cuda().detach()
    else:
        field = Variable(field).cuda().detach()
    field.requires_grad = True
    updates = 0
    for k in reversed(range(ndownsamples)):
      src_ = downsample(k)(src_var).detach()
      src_.requires_grad = False
      src_smooth_mask_ = downsample(k)(src_smooth_mask_var).detach()
      src_smooth_mask_.requires_grad = False
      src_loss_mask_ = downsample(k)(src_loss_mask_var).detach()
      src_loss_mask_.requires_grad = False
      dst_list_, dst_mask_list_ = [], []
      for d, m in zip(dst_vars, dst_mask_vars):
        dst_ = downsample(k)(d).detach()
        dst_.requires_grad = False
        dst_list_.append(dst_)
        mask_ = downsample(k)(m).detach()
        mask_.requires_grad = False
        dst_mask_list_.append(mask_)
      field = field.detach()
      field.requires_grad = True
      opt = torch.optim.SGD([field], lr=lr/(k+1))
      #sched = lr_scheduler.StepLR(opt, step_size=1, gamma=0.995)
      costs = []
      start_updates = updates
      print('Downsample factor {0}x'.format(2**k))
      while True:
        updates += 1
        I = self.get_identity_grid(field.size(2))
        pred = F.grid_sample(src_, field + I)
        pred_mask = F.grid_sample(src_loss_mask_, field + I)
        centered_mask = self.center(src_smooth_mask_.squeeze(0), (1,2), 128/(2**k))
        centered_field = self.center(field, (1,2), 128/(2**k))
        penalty1 = self.penalty([centered_field], centered_mask)
        cost = penalty1 * lambda1/(k+1)
        for d, m in zip(dst_list_, dst_mask_list_):
          mask = torch.mul(pred_mask, m)
          mse = torch.mul(pred - d, mask)**2
          cost += torch.mean(self.center(mse, (-1,-2), 128 / (2**k)))
        #cost = diff + penalty1 * lambda1/(k+1)
        print(cost.data.cpu().numpy())
        costs.append(cost)
        cost.backward()
        opt.step()
        #sched.step()uniform
        opt.zero_grad()
        if len(costs) > avgn + currn and len(costs)>min_iter:
            hist_costs = costs[-(avgn+currn):-currn]
            hist = sum(hist_costs).data[0] / avgn
            curr = sum(costs[-currn:]).data[0] / currn
            if abs((hist-curr)/hist) < eps/(2**k) or len(costs)>max_iter:
                break
        #print downsamples, updates - start_updates
      if write_field:
        self.stack.save('field', self.image_mip+k, self.bbox, field + I, is_field=True)
      if k > 0:
        field = upsample(field.permute(0,3,1,2)).permute(0,2,3,1)
    #print(cost.data[0], diff.data[0], penalty1.data[0])
    print('done:', updates)
    print(field.shape)
    # return self.center(field, (1,2), crop*2).data.cpu().numpy()[0]
    return self.center(field, (1,2), crop*2)

In [75]:
paths = {'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
         'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
         'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
         'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
         'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/refactor_v2',
         'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/refactor_v2/field'}
stack = Stack(paths)
offsets = [Vec(102716, 107077, 526), # fold
           Vec(102320, 111106, 526), # crack
           Vec(102565, 103904, 527)] # no defect
size = Vec(2048, 2048, 1)
bbox = Bbox(offset, offset+size)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 1
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, 
                                                  eps=0.001, min_iter=100, max_iter=5000)
    o.render(field)

In [77]:
paths = {'image': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked',
         'fold': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1',
         'crack': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3',
         'loss': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth',
         'output': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/refactor_v4',
         'field': 'gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/refactor_v4/field'}
stack = Stack(paths)
offsets = [Vec(102716, 107077, 526), # fold
           Vec(102320, 111106, 526), # crack
           Vec(102565, 103904, 527)] # no defect
size = Vec(2048, 2048, 1)
bbox = Bbox(offset, offset+size)
image_mip = 2
fold_mip = 5
crack_mip = 5
loss_mip = 2
output_mip = 2
dst_increment = [-1,1]
for offset in offsets:
    bbox = Bbox(offset, offset+size)
    o = Optimizer(stack, bbox, dst_increment, image_mip, fold_mip, crack_mip, loss_mip, output_mip)
    field = o.process(ndownsamples=5, currn=5, avgn=20, lambda1=1.0, lr=0.1, 
                                                  eps=0.001, min_iter=100, max_iter=5000)
    o.render(field)

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.21it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 12.37it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.41it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.49it/s]

Downloading:   0%|          | 0/4 [00:00<?, ?it/s]
Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [0

Downsample factor 16x
[0.26509356]
[0.252102]
[0.24362797]
[0.23755568]
[0.23254833]
[0.22815262]
[0.22413337]
[0.22028378]
[0.21652883]
[0.21293144]
[0.20935577]
[0.2058531]
[0.20238806]
[0.1989522]
[0.19561543]
[0.19227831]
[0.18907154]
[0.18588397]
[0.18284625]
[0.17982715]
[0.17692098]
[0.17415455]
[0.17148772]
[0.16892251]
[0.16646686]
[0.16413912]
[0.16190696]
[0.15981698]
[0.15778328]
[0.15590337]
[0.1540969]
[0.15242216]
[0.1508867]
[0.14951718]
[0.14818037]
[0.14696266]
[0.14580713]
[0.14464669]
[0.14358965]
[0.1425417]
[0.14157362]
[0.1406411]
[0.13973612]
[0.13884896]
[0.13810447]
[0.13731332]
[0.1365481]
[0.1358766]
[0.1351722]
[0.13452895]
[0.13389328]
[0.1333294]
[0.13277756]
[0.13227807]
[0.13174815]
[0.1312702]
[0.13082023]
[0.13039345]
[0.13004853]
[0.12964806]
[0.12923285]
[0.1288928]
[0.1285827]
[0.1282273]
[0.12792599]
[0.12759218]
[0.12728974]
[0.12697965]
[0.12666951]
[0.12635256]
[0.12612633]
[0.12585205]
[0.12558085]
[0.1253503]
[0.12505892]
[0.12482259]
[0.1245

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.83it/s]



Downsample factor 8x
[0.47248226]
[0.45366576]
[0.43704936]
[0.4220113]
[0.40797883]
[0.39551836]
[0.38462362]
[0.375321]
[0.3673114]
[0.36172497]
[0.35591513]
[0.35193157]
[0.347481]
[0.34412467]
[0.3405043]
[0.33804947]
[0.33529013]
[0.33322635]
[0.33120206]
[0.32945272]
[0.32794425]
[0.32647786]
[0.3250457]
[0.32394174]
[0.32256585]
[0.32181644]
[0.32056075]
[0.3199761]
[0.31816214]
[0.31778038]
[0.31662995]
[0.31630197]
[0.31520075]
[0.31496066]
[0.31396484]
[0.31368908]
[0.3128267]
[0.31258643]
[0.3119261]
[0.3115309]
[0.3111483]
[0.31060797]
[0.31027654]
[0.309816]
[0.30942446]
[0.3091082]
[0.30865717]
[0.3084547]
[0.30781108]
[0.30792046]
[0.30727503]
[0.30727145]
[0.30669686]
[0.30663097]
[0.30615646]
[0.3060486]
[0.30573457]
[0.30540067]
[0.3053125]
[0.30502358]
[0.3046514]
[0.30469745]
[0.30400878]
[0.3043844]
[0.30355024]
[0.3039968]
[0.30315915]
[0.30361888]
[0.30288178]
[0.30311012]
[0.30254942]
[0.30268484]
[0.30227166]
[0.30218726]
[0.30195028]
[0.3016913]
[0.3016448]
[0

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.85it/s]



...

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  6.17it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.68it/s]



Downsample factor 1x
[1.6262764]
[1.263841]
[1.2220063]
[1.2100513]
[1.204066]
[1.2002215]
[1.1973786]
[1.1951983]
[1.1933913]
[1.1918237]
[1.190398]
[1.1891843]
[1.1880264]
[1.1870577]
[1.1861169]
[1.1853249]
[1.18454]
[1.1838624]
[1.1831563]
[1.182636]
[1.182033]
[1.1815245]
[1.1809664]
[1.1805661]
[1.1800468]
[1.1796532]
[1.1791923]
[1.1788876]
[1.1784688]
[1.1781951]
[1.1777802]
[1.1775181]
[1.1771297]
[1.1768785]
[1.1765178]
[1.1763059]
[1.1759472]
[1.1757629]
[1.1754394]
[1.1752737]
[1.1750083]
[1.1748476]
[1.1745596]
[1.1744139]
[1.1741648]
[1.1740239]
[1.1737832]
[1.1736865]
[1.1734252]
[1.1732893]
[1.1731033]
[1.173007]
[1.1728001]
[1.1726823]
[1.1724925]
[1.1723783]
[1.1721928]
[1.1720748]
[1.1719315]
[1.171824]
[1.1717371]
[1.1716549]
[1.1715081]
[1.1714205]
[1.1712998]
[1.1711698]
[1.1710472]
[1.1709704]
[1.1708694]
[1.170778]
[1.1706431]
[1.1705906]
[1.1704916]
[1.170413]
[1.1703142]
[1.1702254]
[1.1701232]
[1.1700602]
[1.1699978]
[1.1699462]
[1.1698755]
[1.1697824]
[1.169

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  4.91it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

done: 1412
torch.Size([1, 512, 512, 2])



Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.28it/s]

